Exercise B

B.1) Calibrate the camera using the built-in functions in OpenCV. Note: the checkerboard used in the images is of size 6x9 with each black/white square having a size of 20 mm.

B.2) Unfortunately not all the lines in the images are straight. Especially towards the edges of the images. Rectify this by undistorting (cv2.undistort) one of the images. Save the undistorted image. Is there a difference between the original and undistorted image?

In [ ]:
# Importing libraries & packages
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import time
import glob

# Images path
data_A = "../Images/exe_a/"
data_B = "../Images/exe_b/"
data_C = "../Images/exe_c/"
output = "output/"

B.1) Calibrate the camera using the built-in functions in OpenCV. Note: the checkerboard used in the images is of size 6x9 with each black/white square having a size of 20 mm.

In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((9*6,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob(data_B+'*.jpeg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (9,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

    # Draw and display the corners
    cv.drawChessboardCorners(img, (9,6), corners2, ret)
    cv.imshow('img', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

# Calibrating camera   
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
print('Camera matrix', mtx)
print('Distortion coeffs', dist)
print('Rotation vectors', rvecs)
print('Translation vectors', tvecs)

B.2) Unfortunately not all the lines in the images are straight. Especially towards the edges of the images. Rectify this by undistorting (cv2.undistort) one of the images. Save the undistorted image. Is there a difference between the original and undistorted image?

In [ ]:
# Load img and refine camera matrix
img = cv.imread(data_B+'00004.jpeg')
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite(output+'calibresult.png', dst)

Exercise C

C.1) In image 'images/00006.jpeg' draw the x, y and z axes such that they correspond with the checkerboard with origo at (0,0,0) in relation to the checkerboard in the image. Each axis should be 40 mm in the world space.

C.2) Try to project the point (40.0, 0.0, 0.0) from the 3D world coordinates (in relation to the checkerboard) to 2D image coordinates without using the cv2.projectPoints function and instead apply the matrices K[R|t] (which is known from our earlier calibration). Draw the resulting point onto the original image and save the resulting. Does it look like expected?

C.1) In image 'images/00006.jpeg' draw the x, y and z axes such that they correspond with the checkerboard with origo at (0,0,0) in relation to the checkerboard in the image. Each axis should be 40 mm in the world space.

In [ ]:
# Function to draw 3D axis
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    origo = (int(imgpts[0][0][0]),
            int(imgpts[0][0][1]))
    img = cv.line(img, corner, origo, (255,0,0), 5)
    img = cv.line(img, corner, origo, (0,255,0), 5)
    img = cv.line(img, corner, origo, (0,0,255), 5)
    return img

In [ ]:
# Create termination criteria , obj points + axis points
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

for fname in glob.glob(data_C+'00006.jpeg'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, (9,6),None)
    if ret == True:
        corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
        # project 3D points to image plane
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)
        img = draw(img,corners2,imgpts)
        cv.imshow('img',img)
        k = cv.waitKey(0)
        if k == ord('s'):
            cv.imwrite(output+fname[:6]+'.png', img)
cv.destroyAllWindows()

C.2) Try to project the point (40.0, 0.0, 0.0) from the 3D world coordinates (in relation to the checkerboard) to 2D image coordinates without using the cv2.projectPoints function and instead apply the matrices K[R|t] (which is known from our earlier calibration). Draw the resulting point onto the original image and save the resulting. Does it look like expected?